In [34]:
import numpy as np
import pandas as pd

In [35]:
df_bak = pd.read_excel('../01-Data/ordens_pre_process_revisado.xlsx')

In [36]:
df_caixas_tipo = pd.read_excel('../01-Data/Dados - desafio disciplina RPVMM.xlsx', sheet_name='Tipos de caixa')

In [37]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']
df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [38]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [39]:
df = df_bak.drop(['Unnamed: 0'], axis=1)
df.columns = df.columns.str.replace(' ', '_')
df["Volume_total"] = df["Volume_unit"] * df["Peças"]
df["Peso_total"] = df["Peso_unit"] * df["Peças"]

In [40]:
# df = df.query("chave_loja_buffer_onda in ('10_EN_3', '28_EN_4', '16_EN_3', '2_EN_3', '31_EN_3', '33_EN_3', '8_EN_4', '10_EN_4', '9_EN_4', '3_EN_3', '25_EN_4', '18_EN_4', '48_EN_4', '8_EN_3', '10_EN_1', '23_EN_3', '11_EN_4', '3_EN_4', '2_EN_4', '9_EN_3')")
df = df.query('chave_loja_buffer_onda in ("16_EN_3","9_EN_4","33_EN_3")')

In [41]:
print(df['Inner'].unique())

[1]


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1041 entries, 33868 to 59790
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Ordem_de_produção          1041 non-null   int64         
 1   Data_da_ordem_de_produção  1041 non-null   datetime64[ns]
 2   Produto                    1041 non-null   int64         
 3   Comprimento_unit           1041 non-null   int64         
 4   Largura_unit               1041 non-null   int64         
 5   Altura_unit                1041 non-null   int64         
 6   Peso_unit                  1041 non-null   float64       
 7   Peças                      1041 non-null   int64         
 8   Caixa_padrão               1041 non-null   object        
 9   Loja                       1041 non-null   int64         
 10  Classe_de_onda             1041 non-null   int64         
 11  Tipo_de_buffer             1041 non-null   object        
 12  Item_p

In [43]:
df["Peças"].sum()   

5343

In [44]:
def first_fit_decreasing(df, df_restrictions):
    # Sort items in decreasing order of volume
    df_sorted = df.sort_values(by='Volume_total', ascending=False)
    
    # Initialize bins list
    bins = []
    
    # Flag to track if a split was made
    split_made = True
    items_to_drop = []
    df_sorted = df_sorted.reset_index(drop=True)
    items_to_add = []

      
    while len(df_sorted) != 0:
        split_made = False

        df_sorted = df_sorted[~df_sorted.index.isin(items_to_drop)]

        # Add needed items to df_sorted
        if len(items_to_add) != 0:
            df_sorted = pd.concat([df_sorted, pd.DataFrame(items_to_add)])
        items_to_add = []
        
        df_sorted = df_sorted.reset_index(drop=True)
        items_to_drop = []
        
        # Make a copy of the sorted DataFrame
        df_sorted_copy = df_sorted.copy()
        # Iterate over items in the copy of the sorted DataFrame
        for index, item in df_sorted_copy.iterrows():
            # Flag to check if the item is placed
            placed = False
            
            # Try to place the item in existing bins
            for bin in bins:
                if (bin['Volume'] + item['Volume_total']) <= bin["vol_limit"] \
                   and (bin['Peso_total'] + item['Peso_total']) <= 23 \
                   and bin['Amt.Items'] + item['Peças'] <= bin["item_limit"] \
                   and item["Loja"] == bin["Loja"]:
                    bin['Items'].append(index)
                    bin['Volume'] += item['Volume_total']
                    bin['Peso_total'] += item['Peso_total']
                    bin['Amt.Items'] += item['Peças']

                    # Remove the item from the original DataFrame
                    items_to_drop.append(index)

                    placed = True
                    break
            
            # If the item couldn't be placed entirely, reduce it by "inner" means until it fits into a bin
            while not placed and item['Peças'] > item['Inner']:
                # Reduce the item to its inner size
                detached_portion = item.copy()
                detached_portion['Peças'] = item['Inner']
                detached_portion['Volume_total'] = item['Volume_unit'] * item['Inner']
                detached_portion['Peso_total'] = item['Peso_unit'] * item['Inner']
                
                # Update the remaining portion of the item
                item['Peças'] -= item['Inner']
                item['Volume_total'] -= item['Volume_unit'] * item['Inner']
                item['Peso_total'] -= item['Peso_unit'] * item['Inner']
                
                # Concatenate the reduced portion back to the original DataFrame
                items_to_add.append(detached_portion)
                split_made = True
                
                # Try to place the reduced item in existing bins
                for bin in bins:
                    if (bin['Volume'] + item['Volume_total']) <= bin["vol_limit"] \
                       and (bin['Peso_total'] + item['Peso_total']) <= 23 \
                       and bin['Amt.Items'] + item['Peças'] <= bin["item_limit"] \
                       and item["Loja"] == bin["Loja"]:
                        bin['Items'].append(index)  # Append index of the appended reduced item
                        bin['Volume'] += item['Volume_total']
                        bin['Peso_total'] += item['Peso_total']
                        bin['Amt.Items'] += item['Peças']

                        items_to_drop.append(index) # Remove the reduced item from the original DataFrame
                        placed = True
                        break
            
            # If the item still couldn't be placed, create a new bin
            if not placed:
                bins.append({'Items': [index], 'Volume': item['Volume_total'], 'Peso_total': item['Peso_total'], 'Amt.Items': item['Peças'], "Loja": item["Loja"], "vol_limit": df_restrictions[df_restrictions["Tipo_de_buffer"] == item["Tipo_de_buffer"]]["volume_util_caixa"].values[0], "item_limit": df_restrictions[df_restrictions["Tipo_de_buffer"] == item["Tipo_de_buffer"]]["Peças_max"].values[0], })
                items_to_drop.append(index)
                placed = True
                break
    
    return bins


In [22]:
def old_first_fit_decreasing(df, bin_capacity_vol, bin_capacity_weight, bin_item_limit):
    # Sort items in decreasing order of volume
    df_sorted = df.sort_values(by='Volume_total', ascending=False)
    
    # Initialize bins list
    bins = []
    
    for index, item in df_sorted.iterrows():
        for bin in bins:
            if (bin['Volume'] + item['Volume_total']) <= bin_capacity_vol and \
                (bin['Peso_total'] + item['Peso_total']) <= bin_capacity_weight and \
                bin['Amt.Items'] <= bin_item_limit:
                    bin['Items'].append(index)
                    bin['Volume'] += item['Volume_total']
                    bin['Peso_total'] += item['Peso_total']
                    bin['Amt.Items'] += item['Peças']
                    break
        else:
            # Create new bin
            bins.append({'Items': [index], 'Volume': item['Volume_total'], 'Peso_total': item['Peso_total'], 'Amt.Items': item['Peças']})
    
    return bins



In [45]:
bin_capacity_vol = 75538008
bin_capacity_weight = 23
bin_item_limit = 512
packed_bins_new = first_fit_decreasing(df, df_restrictions=df_caixas_padrao)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins_new):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")


Packed bins (new method):
Bin 1: Amt. Items 12 (Total Volume: 75405854 cubic mm | Total Weight: 3.5100kg)
Bin 2: Amt. Items 7 (Total Volume: 75471286 cubic mm | Total Weight: 3.8500kg)
Bin 3: Amt. Items 14 (Total Volume: 75384444 cubic mm | Total Weight: 3.9700kg)
Bin 4: Amt. Items 11 (Total Volume: 75416898 cubic mm | Total Weight: 3.5670kg)
Bin 5: Amt. Items 11 (Total Volume: 75135020 cubic mm | Total Weight: 5.1340kg)
Bin 6: Amt. Items 8 (Total Volume: 75398001 cubic mm | Total Weight: 4.2150kg)
Bin 7: Amt. Items 7 (Total Volume: 75390265 cubic mm | Total Weight: 3.9070kg)
Bin 8: Amt. Items 8 (Total Volume: 75509686 cubic mm | Total Weight: 3.9950kg)
Bin 9: Amt. Items 8 (Total Volume: 75421764 cubic mm | Total Weight: 4.0160kg)
Bin 10: Amt. Items 20 (Total Volume: 75293514 cubic mm | Total Weight: 3.9610kg)
Bin 11: Amt. Items 9 (Total Volume: 74527265 cubic mm | Total Weight: 4.3330kg)
Bin 12: Amt. Items 9 (Total Volume: 75052194 cubic mm | Total Weight: 4.6350kg)
Bin 13: Amt. Items

In [24]:
packed_bins_old = old_first_fit_decreasing(df, bin_capacity_vol, bin_capacity_weight, bin_item_limit)
print("\nPacked bins (old method):")
for i, bin in enumerate(packed_bins_old):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")


KeyboardInterrupt: 